 \\[
 \begin{aligned}
 \frac{dS}{dt} &= -\frac{\beta SI}{N} \\
 \frac{dI}{dt} &= \frac{\beta SI}{N} - \gamma I\\
 \frac{dR}{dt} &= \gamma I 
 \end{aligned}
 \\]


In [1]:
import numpy as np
from scipy.optimize import minimize
import plotly.graph_objects as go 
import plotly.express as px 
import plotly.io as pio
from plotly.subplots import make_subplots

In [2]:
def binomial(trial, density):
    return np.random.binomial(trial, density)

def poisson(trial, density):
    return np.random.poisson(trial*density)

In [3]:
def loglikely(model, obs):
    a, b = model, obs
    lamb = a.I[b.ts] / a.N * b.ns
    return -np.sum(lamb) + np.dot(b.xs, np.log(lamb)) 

def likely(model, obs):
    return np.exp(loglikely(model, obs))

In [30]:
class SIR_Model:
    def __init__(self, beta=1, gamma=1, dt=1):
        self.beta = beta*dt
        self.gamma = gamma*dt
        self.R0 = beta / gamma
        
    def __str__(self):
        return "β=" + str(self.beta) + " γ=" + str(self.gamma)
    
    def inhabit(self, S0, I0, R0):
        self.S0 = S0
        self.I0 = I0
        self.R0 = R0
        self.N = S0 + I0 + R0
        
    def epidemic(self, T):
        S = np.zeros(T+1)
        I = np.zeros(T+1)
        R = np.zeros(T+1)
        S[0] = self.S0
        I[0] = self.I0
        R[0] = self.R0

        for t in range(T):
            a, b = self.beta*S[t]*I[t]/self.N, self.gamma*I[t]
            S[t+1] = S[t] - a
            I[t+1] = I[t] + a - b
            R[t+1] = R[t] + b
            
        self.S = S
        self.I = I
        self.R = R
        self.T = T
        
    def plot(self):
        fig = go.Figure()
        fig.update_layout(
            margin=dict(b=0, l=0, r=0, t=50)
        )
        T = self.T
        fig.add_scatter(x=np.arange(T+1), y=self.S.astype(int), name="Susceptible", hovertemplate="%{y}")
        fig.add_scatter(x=np.arange(T+1), y=self.I.astype(int), name="Infectious", hovertemplate="%{y}")
        fig.add_scatter(x=np.arange(T+1), y=self.R.astype(int), name="Removed", hovertemplate="%{y}")
        return fig
    
    def sample(self, time, export, f):
        for t, e in zip(time, export):
            yield f(e, self.I[t]/self.N)
            
    def infer_obs_map(self, obs):
        def fun(x):
            m = SIR_Model(x[0], x[1])
            m.inhabit(self.S0, self.I0, self.R0)
            m.epidemic(self.T)
            return (-loglikely(m, obs))
        
        res = minimize(fun, (0.5,0.5), method='nelder-mead', options={'xatol': 1e-8, 'disp': True})
        return res.x
    
    def infer_obs_mcmc(self, obs):
        def fun(x):
            m = SIR_Model(x[0], x[1])
            m.inhabit(self.S0, self.I0, self.R0)
            m.epidemic(self.T)
            return likely(m, obs)
        
        N = 1000
        walk = np.zeros((N+1, 2))
        x = np.array([0.5, 0.5])
        px = fun(x)
        walk[0, :] = x
        for n in range(N):
            y = x + np.random.multivariate_normal((0, 0), [[0.01, 0],[0, 0.01]])
            if y[0]<=0 or y[0]>=1 or y[1]<=0 or y[1]>=1:
                y = x
            py = fun(y)
            if np.random.rand() < py/px:
                x, px = y, py
            walk[n+1, :] = x
        return np.mean(walk[100:, :], axis=0), walk

In [31]:
a = SIR_Model(3, 1, 0.1)
a.inhabit(1e7-1e4, 1e4, 0)

T = 100
a.epidemic(T)

np.random.seed(0)
b_design = np.arange(T//10, T, T//10)
export = 10*np.ones_like(b_design)
positive = np.array(list(a.sample(b_design, export, poisson)))

fig = a.plot()
fig.add_scatter(x=b_design, y=positive/export*a.N, mode="markers", name="Guessed", hovertemplate="%{y}")

In [32]:
class Obs:
    def __init__(self, ts, ns, xs):
        self.ts = ts
        self.ns = ns
        self.xs = xs

In [33]:
b = Obs(b_design, export, positive)

In [34]:
x, y = np.logspace(-2, 0, 20), np.logspace(-2, 0, 20)
z = np.zeros((len(y), len(x)))
for i in range(len(y)):
    for j in range(len(x)):
        m = SIR_Model(x[j], y[i])
        m.inhabit(a.S0, a.I0, a.R0)
        m.epidemic(100)
        z[len(x)-1-i, j] = loglikely(m, b)

In [35]:
np.log(x)

array([-4.60517019, -4.36279281, -4.12041543, -3.87803805, -3.63566067,
       -3.39328329, -3.15090592, -2.90852854, -2.66615116, -2.42377378,
       -2.1813964 , -1.93901903, -1.69664165, -1.45426427, -1.21188689,
       -0.96950951, -0.72713213, -0.48475476, -0.24237738,  0.        ])

In [48]:
fig = go.Figure(data=go.Contour(z=np.log(-z), x=np.log(x), y=np.log(y)))
fig.show()

In [49]:
res, walk = a.infer_obs_mcmc(b)
res

array([0.34355863, 0.09339253])

In [50]:
fig.add_scatter(x=np.log(walk[:, 0]), y=np.log(walk[:, 1]), mode="markers", opacity=0.7)

In [39]:
a.infer_obs_map(b)

Optimization terminated successfully.
         Current function value: 2.487482
         Iterations: 68
         Function evaluations: 132


array([0.32905169, 0.08998246])

In [40]:
a.beta, a.gamma = res[0], res[1]

In [41]:
np.log(-loglikely(a, b))

1.2094653016504042